In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input/dev-dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

../input/dev-dataset/vinted_dev_val/women__clothes__coats_and_jackets__coats__parkas/1399264635.jpeg
../input/dev-dataset/vinted_dev_val/women__clothes__coats_and_jackets__coats__parkas/1406833246.jpeg
../input/dev-dataset/vinted_dev_val/women__clothes__coats_and_jackets__coats__parkas/1622924703.jpeg
../input/dev-dataset/vinted_dev_val/women__clothes__coats_and_jackets__coats__parkas/1533191749.jpeg
../input/dev-dataset/vinted_dev_val/women__clothes__coats_and_jackets__coats__parkas/1798298987.jpeg
../input/dev-dataset/vinted_dev_val/women__clothes__coats_and_jackets__coats__parkas/1386460948.jpeg
../input/dev-dataset/vinted_dev_val/women__clothes__coats_and_jackets__coats__parkas/1434262389.jpeg
../input/dev-dataset/vinted_dev_val/women__clothes__coats_and_jackets__coats__parkas/1400160131.jpeg
../input/dev-dataset/vinted_dev_val/women__clothes__coats_and_jackets__coats__parkas/1378609906.jpeg
../input/dev-dataset/vinted_dev_val/women__clothes__coats_and_jackets__coats__parkas/137170

In [2]:
! yes | pip install -q lightning-flash 'lightning-flash[image]'

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
beatrix-jupyterlab 3.1.7 requires google-cloud-bigquery-storage, which is not installed.
thinc 8.0.15 requires typing-extensions<4.0.0.0,>=3.7.4.1; python_version < "3.8", but you have typing-extensions 4.2.0 which is incompatible.
spacy 3.2.4 requires typing-extensions<4.0.0.0,>=3.7.4; python_version < "3.8", but you have typing-extensions 4.2.0 which is incompatible.
gcsfs 2022.2.0 requires fsspec==2022.02.0, but you have fsspec 2022.3.0 which is incompatible.
cloud-tpu-client 0.10 requires google-api-python-client==1.8.0, but you have google-api-python-client 1.12.11 which is incompatible.
yes: standard output: Broken pipe


In [3]:
import torch

import flash
from flash.core.data.utils import download_data
from flash.image import ImageClassificationData, ImageClassifier

# 1. Create the DataModule
# download_data("../input/dev-dataset", "./data")

datamodule = ImageClassificationData.from_folders(
    train_folder="../input/dev-dataset/vinted_dev_train",
    val_folder="../input/dev-dataset/vinted_dev_val",
    batch_size=4,
    transform_kwargs={"image_size": (100, 100), "mean": (0.485, 0.456, 0.406), "std": (0.229, 0.224, 0.225)},
)

# datamodule_test = ImageClassificationData.from_folders(
# #     train_folder="../input/dev-dataset/vinted_dev_train",
#     val_folder="../input/dev-dataset/vinted_dev_val",
#     batch_size=4,
#     transform_kwargs={"image_size": (100, 100), "mean": (0.485, 0.456, 0.406), "std": (0.229, 0.224, 0.225)},
# )

In [4]:
# 2. Build the task
model = ImageClassifier(backbone="mobilenetv3_small_100", labels=datamodule.labels)

In [5]:
# 3. Create the trainer and finetune the model
trainer = flash.Trainer(max_epochs=1, gpus=torch.cuda.device_count())
trainer.finetune(model, datamodule=datamodule, strategy="freeze")

Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [6]:
# 4. Predict what's on a few images! ants or bees?
datamodule = ImageClassificationData.from_files(
    predict_files=[
        "../input/dev-dataset/vinted_dev_val/women__clothes__coats_and_jackets__coats__duster_coats/1375442944.jpeg",
        "../input/dev-dataset/vinted_dev_val/women__clothes__coats_and_jackets__coats__duster_coats/1406737243.jpeg",
        "../input/dev-dataset/vinted_dev_val/women__clothes__coats_and_jackets__coats__duster_coats/1486553230.jpeg",
    ],
    batch_size=3,
)
predictions = trainer.predict(model, datamodule=datamodule, output="labels")
print(predictions)

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Predicting: 179it [00:00, ?it/s]

[['women__clothes__coats_and_jackets__coats__duster_coats', 'women__clothes__coats_and_jackets__coats__duster_coats', 'women__clothes__coats_and_jackets__coats__duster_coats']]


###### 5. Save the model!
trainer.save_checkpoint("image_classification_model.pt")